Kemal Demirel

191104091

HW2- SORU2

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 1-Load a gray scale image and a template

Load a gray scale image

In [ ]:
gray_scale_image = cv2.imread('connectors.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(gray_scale_image, cmap='gray')
plt.show()

Load a template

In [ ]:
template = cv2.imread('connector_pattern.png', 0)
plt.imshow(template, cmap='gray')
plt.show()

## 2- Implement a correlation

In [ ]:
def correlation(image, template):
    img_h, img_w = image.shape
    tpl_h, tpl_w = template.shape
    
    output = np.zeros((img_h, img_w))
    
    for y in range(img_h - tpl_h + 1):
        for x in range(img_w - tpl_w + 1):
            # Compute correlation for the current position
            corr = np.sum(template * image[y:y+tpl_h, x:x+tpl_w])
            output[y, x] = corr
    return output    

Correlation is computed

In [ ]:
result_correlation = correlation(gray_scale_image, template)
plt.imshow(result_correlation, cmap='gray')
plt.title("Image correlation")
plt.show()

## 3- Implement Zero-mean correlation

In [ ]:
def zero_mean_correlation(image, template):
    template_mean = np.mean(template)
    template_zero_mean = template - template_mean
    image_pad = np.pad(image, ((template.shape[0]//2,template.shape[0]//2), (template.shape[1]//2,template.shape[1]//2)), mode='constant')
    output = np.zeros_like(image)
    for row in range(output.shape[0]):
        for col in range(output.shape[1]):
            patch = image_pad[row:row+template.shape[0], col:col+template.shape[1]]
            patch_mean = np.mean(patch)
            zm_correlation = np.sum((template_zero_mean * (patch - patch_mean)))
            output[row, col] = zm_correlation
    return output

## 4- Implement Sum Square Difference:

In [ ]:
def sum_square_difference(image, template):
    output = np.zeros_like(image)
    for row in range(output.shape[0]):
        for col in range(output.shape[1]):
            patch = image[row:row+template.shape[0], col:col+template.shape[1]]
            ssd = np.sum((template - patch)**2)
            output[row, col] = ssd

    return output


## 5- Implement Normalized Cross Correlation:

In [ ]:
def normalized_cross_correlation(image, template):
    output = np.zeros_like(image)
    template_mean = np.mean(template)
    template_std = np.std(template)

    for row in range(output.shape[0]):
        for col in range(output.shape[1]):
            patch = image[row:row+template.shape[0], col:col+template.shape[1]]
            patch_mean = np.mean(patch)
            patch_std = np.std(patch)
            ncc = np.sum((template - template_mean) * (patch - patch_mean)) / (template_std * patch_std)
            output[row, col] = ncc

    return output

## 6- Apply your template matching functions to the following images and templates: 

A grayscale image with a rectangular template

In [ ]:
corr_output = zero_mean_correlation(gray_scale_image, template)
ssd_output = sum_square_difference(gray_scale_image, template)
ncc_output = normalized_cross_correlation(gray_scale_image, template)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].imshow(gray_scale_image, cmap='gray')
axs[0, 0].set_title('Original Image')
axs[0, 1].imshow(template, cmap='gray')
axs[0, 1].set_title('Template')
axs[1, 0].imshow(corr_output, cmap='gray')
axs[1, 0].set_title('Zero-Mean Correlation')
axs[1, 1].imshow(ssd_output, cmap='gray')
axs[1, 1].set_title('Sum Square Difference')
plt.show()

A grayscale image with a circular template

In [ ]:
rows, cols = template.shape
center_row, center_col = rows//2, cols//2
radius = 60
y, x = np.ogrid[-center_row:rows-center_row, -center_col:cols-center_col]
mask = x*x + y*y <= radius*radius
template[~mask] = 0
corr_output = zero_mean_correlation(gray_scale_image, template)
ssd_output = sum_square_difference(gray_scale_image, template)
ncc_output = normalized_cross_correlation(gray_scale_image, template)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].imshow(gray_scale_image, cmap='gray')
axs[0, 0].set_title('Original Image')
axs[0, 1].imshow(template, cmap='gray')
axs[0, 1].set_title('Template')
axs[1, 0].imshow(corr_output, cmap='gray')
axs[1, 0].set_title('Zero-Mean Correlation')
axs[1, 1].imshow(ssd_output, cmap='gray')
axs[1, 1].set_title('Sum Square Difference')
plt.show()

## 7- After applying the template matching function, you will have an output image where each pixel represents the similarity measure between the template and the image at that location. To identify the matches, you can apply a threshold to the output image and then find the local maxima.

In [ ]:
def find_matches(image, template, threshold, method):
    # Apply template matching function
    if method == 'corr':
        output = zero_mean_correlation(image, template)
    elif method == 'ssd':
        output = sum_square_difference(image, template)
    elif method == 'ncc':
        output = normalized_cross_correlation(image, template)
    else:
        raise ValueError('Invalid matching method specified')

    # Threshold the output image to identify potential matches
    thresholded_output = np.zeros_like(output)
    thresholded_output[output > threshold] = 255

    # Find local maxima in the thresholded image
    local_maxima = np.zeros_like(thresholded_output)
    h, w = thresholded_output.shape[:2]
    for i in range(1, h-1):
        for j in range(1, w-1):
            if thresholded_output[i, j] > 0:
                patch = thresholded_output[i-1:i+2, j-1:j+2]
                if np.max(patch) == thresholded_output[i, j]:
                    local_maxima[i, j] = 255

    # Find the positions of the local maxima and record them as matches
    matches = []
    contours, _ = cv2.findContours(local_maxima.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        moments = cv2.moments(contour)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])
            cy = int(moments['m01'] / moments['m00'])
            matches.append((cx, cy))

    # Overlay rectangles or circles on the original image at the positions of the matches
    output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    if template.shape[0] > 1 and template.shape[1] > 1:
        for match in matches:
            if method == 'corr':
                cv2.rectangle(output_image, (match[0], match[1]), (match[0]+template.shape[1], match[1]+template.shape[0]), (0, 255, 0), 2)
            else:
                cv2.circle(output_image, (match[0], match[1]), int(template.shape[1]/2), (0, 255, 0), 2)
    else:
        for match in matches:
            cv2.drawMarker(output_image, (match[0], match[1]), (0, 255, 0), cv2.MARKER_CROSS, 10, 2)

    return output, thresholded_output, local_maxima, output_image, matches